# Nova Feature para um Site

* Um portal de notícias implementou um play automático de vídeos em suas páginas, visando manter seus usuários por mais tempo no site.
* Analise os dados históricos contendo os acessos do último mês, e mostre se houve um aumento significativo de tempo de navegação após a implementação da feature.
* Arquivos: 
  * Populacao_tempo.csv
  * Amostra_tempo.csv
*  Sabendo que o custo por visualização desse play é de 0.005 centavos, e que cada minuto adicional de navegação gera em média 5 centavos de lucro, defina se essa nova feature deve ser mantida ou não.


In [39]:
# setup
import scipy as sp
from scipy import stats

import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls

py.sign_in('phboueke', '')

In [31]:
# loading data
population = []
sample = []

with open('data/populacao_tempo.csv', encoding="utf-8") as file:
    file.readline()
    for line in file:
        time = float(line.split(';')[1][:-1])
        population.append(time)

        
population = sp.array(population)
popMean = sp.mean(population)
popSd = sp.std(population)

with open('data/amostra_tempo.csv', encoding="utf-8") as file:
    file.readline()
    for line in file:
        time = float(line.split(';')[1][:-1])
        sample.append(time)

sample = sp.array(sample)
sampleMean = sp.mean(sample)

Iremos fazer um teste de hipótese para descobrir se a feature deve ser mantida avaliando os tempos de permanência antes da alteração (population) e depois (sample). A hipótese que queremos decidir primeiramente é a de que a feature impactou nos tempos de uso do site.

In [32]:
result = stats.ttest_ind(population, sample)
print(result)
mean_diff = sampleMean - popMean
print("Mean diff: " + str(mean_diff))

Ttest_indResult(statistic=-23.325754825312675, pvalue=2.6163856353001462e-120)
Mean diff: 1.47376129705


O valor-p obtido nos permite rejeitar H0. Vamos agora calcular a viabilidade financeira da feature sabendo que ela impactou positivamente o tempo de permanencia médio no site.

In [33]:
income_min = 0.05
cost_use = 0.005

gain = (mean_diff * income_min) - cost_use

print(gain)

0.0686880648525


Ok. A feature deve ser mantida, visto que calculamos um valor de lucro esperado de quase 7 centavos! Além desse fato, podemos fazer um box plot para ter uma boa ideia da distribuição dos tempos antes e depois das mudanças.

In [36]:
popsample = np.random.choice(population, len(sample))

before = go.Box(
    y=popsample
)
after = go.Box(
    y=sample
)
data = [before, after]
#py.iplot(data)
tls.embed("https://plot.ly/~phboueke/86/") #use this to embed the graph at the static page

# Clicks do Site

* Uma métrica comum em sites de e-commerce é o número de clicks que um usuário efetua durante a navegação. Um grupo de marketing quer fazer uma campanha de um novo produto, entretanto não sabe se apresenta ele apenas na página ou como um pop-up, que toma a conta de toda a tela. Visando responder esse problema, dois grupos foram selecionados. Para o primeiro grupo, foi apresentado apenas a tela com o produto. Para o segundo, foi apresentado a tela com o pop-up.
* Utilize técnicas estatísticas para informar se faz diferença utilizar o pop-up ou não.
* Arquivos:
  * amostra_A_click.csv
  * amostra_B_click.csv


In [42]:
# loading data
A = []
B = []

with open('data/amostra_A_click.csv', encoding="utf-8") as file:
    file.readline()
    for line in file:
        if "y" in line:
            A.append(1)
        else:
            A.append(0)

with open('data/amostra_B_click.csv', encoding="utf-8") as file:
    file.readline()
    for line in file:
        if "y" in line:
            B.append(1)
        else:
            B.append(0)
            
A = sp.array(A)
A_mean = sp.mean(A)
B = sp.array(B)
B_mean = sp.mean(B)

In [47]:
# let's plot!

trace0 = go.Bar(
    x = ["yes", "no"],
    y = [sum(A), len(A)-sum(A)],
    name = "A",
)
trace1 = go.Bar(
    x = ["yes", "no"],
    y = [sum(B), len(B)-sum(B)],
    name = "B",
)

data = [trace0, trace1]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
#py.iplot(fig)
tls.embed("https://plot.ly/~phboueke/92/")

Sendo o dataset "B" aquele coletado utilizando popup, parece claro que o uso do popup aumentou o número de clicks no produto. Não obstante, faremos o teste da hipótese de que a utilização do popup resultou em diferenças entre os datasets.

In [48]:
result = stats.ttest_ind(A, B)
print(result)
print("A mean: " + str(A_mean) + ", B mean: " + str(B_mean))

Ttest_indResult(statistic=-3.6564760626262478, pvalue=0.00026234731820501704)
A mean: 0.306205493388, B mean: 0.383928571429


O valor-p obtido nos permite afirmar que a utilização do popup resultou em uma diferença estatisticamente significante. A vizualização, por sua vez, nos permite afirmar que essa diferença foi positiva em favor da utilização do popup. Podemos ver, também, por meio das médias calculadas, que os percentuais de clicks e "não clicks" em cada dataset.

# Produtor de Cinema

* Um produtor acredita que deve-se construir um poster para um filme utilizando a maior quantidade de atores possíveis. O fato se deve a uma correlação existente entre a nota no IMDB (imdb_score) e o número de faces existentes nos posters (facenumber_in_poster).
* Verifique a existência dessa correlação e tente ajudar o produtor a conseguir o melhor imdb_score possível.
* Arquivo:
  * movie_metadata.csv


In [81]:
from csv import reader

faces = []
rates = []
#names = []

lines = 0

with open('data/movie_metadata.csv', encoding="utf-8") as file:
    setup = file.readline().split(",")
    for line in reader(file):
        lines += 1
        #ll = line.split(",")
        try:
            faces.append(float(line[15]))
            rates.append(float(line[25]))
            #names.append(line[11])
        except ValueError:
            continue
            
print ("Loaded " + str(len(faces)) + " registers of " + str(lines))

Loaded 5030 registers of 5043


In [103]:
# linear try
slope, intercept, r_value, p_value, std_err = stats.linregress(rates,faces)
line = []
for i in rates:
    line.append(slope*i+intercept)

# polinomial second degree try
z = np.polyfit(rates, faces, 2)
f = np.poly1d(z)
new_rates = np.linspace(np.amin(rates), np.amax(rates), 50)
new_faces = f(new_rates)
    
trace0 = go.Scatter(
    x = rates,
    y = faces,
    #text = names,
    mode = 'markers',
    name = "data"
)
trace1 = go.Scatter(
    x = rates,
    y = line,
    mode = 'lines',
    name = "linear fit"
)
trace2 = go.Scatter(
    x = new_rates,
    y = new_faces,
    mode = 'lines',
    name = "poly fit"
)
tls.embed("https://plot.ly/~phboueke/110/")
#py.iplot([trace0, trace1, trace2])

Não parece haver nenhum tipo de dependência óbvia entre a nota dada ao filme e o número de rostos em seu poster. Vamos calcular a correlação linear entre as duas variáveis.

In [106]:
print("Correlation: " + str(stats.pearsonr(rates, faces)[0]))

Correlation: -0.062957524485


Como foi visto, não parece haver correlação entre as variáveis. Para ajudar o nosso produtor, podemos traçar um mapa de calor relacionando (quase) todas as variáveis numéricas à nota do IMDB e tentar buscar a mais correlacinada.

In [162]:
# creating a matrix where each non null row is a numeric column from the input file
mat = []
lines = 0
with open('data/movie_metadata.csv', encoding="utf-8") as file:
    setup = file.readline().split(",")
    for i in range(len(setup)):
        mat.append([])
    for line in reader(file):
        lines += 1
        tmp = []
        for i in range(len(line)):
            try:
                tmp.append(float(line[i]))
            except ValueError:
                tmp.append(0)
                continue
        #if no errors, commit changes
        for i in range(len(tmp)):
            mat[i].append(tmp[i])
    print ("Columns: " + str(len(setup)) + ", lines: " + str(lines))


Columns: 28, lines: 5043


In [210]:
import math
# calculating all correlations                
relations = {}
it = 0
for i in range(len(mat)):
    if len(mat[i]) != 0:
        relations[i] = [0]*len(mat)
    
for i in range(len(mat)):
    if len(mat[i]) == 0:
        continue
    for j in range(len(mat)):
        if i < j:
            continue
        if len(mat[j]) == 0:
            continue
        rel = stats.pearsonr(mat[i], mat[j])[0]
        relations[i][j] = rel
        relations[j][i] = rel

# list of usable columns
ids = []
for i in range(len(mat)):
    for j in range(len(mat)):
        if not math.isnan(relations[i][j]):
            if j not in ids:
                ids.append(j)

#clean relations
rel = {}
for i in ids:
    rel[i] = [0]*len(ids)
    it = 0
    for j in ids:
        rel[i][it] = relations[i][j]
        it += 1

In [221]:
#prepare for the plot
z = []
for i in ids:
    z.append(rel[i])
        
labels = []
for i in ids:
    labels.append("column" + str(i))
        
trace = go.Heatmap(
    z=z,
    x=labels,
    y=labels
)
#py.iplot([trace])
tls.embed("https://plot.ly/~phboueke/118/")

Agora podemos vizualizar facilmente as correlações entre todas as colunas numéricas. Sabendo que **a nota no IMDB é representada pela coluna 25**, podemos observar que ela apresenta maior correlação com a coluna 12 que, em nosso dataset, é a coluna "num_voted_users". Observamos logo que essas colunas podem estar ligeiramente mais relacionadas por se tratarem especificamente do IMDB. Filmes com melhores classificações atraem mais votos por serem bons. Podemos concluir, portanto, que o nosso produtor não encontrará correlações interessantes que o poderão ajudar nesse dataset.